In [ ]:
import numpy as np

In [55]:
class Piece():
    def __init__(self, skirt, heights):
        self.skirt = np.array(skirt)
        self.heights = np.array(heights)
        self.width = self.heights.size

class Board():
    def __init__(self):
        self.width = 10
        self.height = 20
        self.state = np.zeros((self.height, self.width), dtype=int)
        self.tops = np.zeros(self.width, dtype=int)

    def drop(self, piece, xOffset):
        temp = []
        for i in range(piece.width):
            temp.append(self.tops[xOffset + i] - piece.skirt[xOffset + i])

        tempMax = max(temp)
        for i in range(len(temp)):
            temp[i] = tempMax - temp[i]

        for i in range(len(temp)):
            temp[i] += self.tops[xOffset + i]

        for j in range(piece.width):
            for i in range(piece.heights[j]):
                if i + temp[j] > self.height:
                    raise Exception("game over")
                self.state[i + temp[j]][j + xOffset] = 1
            self.tops[xOffset + j] = temp[j] + piece.heights[j]

    def clearLine(self):
        count = 0
        for i in range(self.height):
            if 0 not in self.state[i]:
                self.state[i:-1] = self.state[i + 1:]
                self.state[-1] = np.zeros(self.width)
                count += 1


    def printState(self):
        for i in self.state[::-1]:
            for j in i:
                print(j, end=' ')
            print()

In [56]:
oPiece = Piece([0, 0], [2, 2])
lPiece = Piece([0, 0], [3, 1])

In [57]:
b = Board()

b.drop(lPiece, 0)
b.printState()

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 
1 1 0 0 0 0 0 0 0 0 


In [58]:
b.drop(oPiece, 0)
b.printState()

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
1 1 0 0 0 0 0 0 0 0 
1 1 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 
1 1 0 0 0 0 0 0 0 0 


In [62]:
temp = [1, 2, 3, 4, 5]
temp[1:-1] = temp[1 + 1:]
temp

[1, 3, 4, 5, 5]

In [63]:
temp = [1, 2, 3, 4, 5]

for i in range(len(temp)):
    print(i)
    if i == 3:
        temp[i] = 0
        i -= 1

0
1
2
3
4


In [ ]:
offset = 3
width = 3

for i in range(offset, offset+width):
    print(i)

temp = np.array([1, 2, 3])
temp[0]

In [ ]:
2 0
0 3

0 3

1 0 1
1 1 2

0 1 1

bottom - top
if +ve, offset
if -ve, hole